# This is the notebook for my Coursera IBM Datascience Capstone project

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


To fix: convert values in the selected features to be a float.

# Introduction/Business Problem

I am looking to determine the probability of a car accident occuring as well as the severity of the accident. I will be utilizing weather, traffic, and road contidion data in order to determine these two factors. My target audience will be car drivers who would like to know this information in order to be safer on the road.

# Data

I will be using the data from the csv provided for my predictions. I will definately be utilizing the road contidtions, weather, and severity. I will be utilizing AI trained on the sample data in order to predict the results. A simple at a glance example would be if it is dark outside, in weather that has bad visibility, at night (which also has bad visiblity), with wet/icy road contitions, then the results should show a high probability of an accident, with the severity being either "Property damage only" or "Injury Collision"

# PLAYING WITH RANDOM THINGS, WILL BE FIXED LATER

In [2]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

print('Done!')

Done!


In [4]:
!conda install -c anaconda seaborn -y
import seaborn as sns
print('Seaborn imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - seaborn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |           py36_0         160 KB  anaconda
    ca-certificates-2020.7.22  |                0         132 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    seaborn-0.10.1             |             py_0         160 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following packages will be UPDATED:

    ca-certificates: 2020.6.24-0        --> 2020.7.22-0       anaconda
    certifi:         2020.6.20-py36_0   --> 2020.6.20-py36_0  anaconda
    openssl:         1.1.1g-h7b6447c_0  --> 1.1.1g-h7b6447c_0 anaconda
    seaborn:         0.9.0-pyh91

In [3]:
!wget -O collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

--2020-08-28 18:15:30--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘collisions.csv’

100%[======================================>] 73,917,638  45.7MB/s   in 1.5s   

2020-08-28 18:15:31 (45.7 MB/s) - ‘collisions.csv’ saved [73917638/73917638]



In [ ]:
df = pd.read_csv('collisions.csv')
df.head()

In [6]:
df.shape

NameError: name 'df' is not defined

### Data Visualization and Preprocessing

In [7]:
df.columns

NameError: name 'df' is not defined

In [8]:
df.shape

NameError: name 'df' is not defined

In [9]:
dfSev = df['SEVERITYDESC'].value_counts()
dfSev

NameError: name 'df' is not defined

## Property Damage Only vs Injury Collision

In [10]:
#set the colors for collision for the following charts
pdoCol = '#FFC416'
injCol = '#FF3B00'
collcol = (pdoCol, injCol)

In [11]:
dfSevP = dfSev.plot(kind = 'pie',
                    figsize = (5,6),
                    autopct='%1.1f%%',
                    startangle = 90,
                    colors = collcol,
                    shadow = True
                   )

NameError: name 'dfSev' is not defined

As we can see from this simple pie chart, in most collisions there is only propertjy damage compared to injuries by a fairly significant margin

## Light and Injury

In [12]:
df['LIGHTCOND'].value_counts()

NameError: name 'df' is not defined

In [13]:
lc = df.groupby(['LIGHTCOND'])['SEVERITYDESC'].value_counts()
lc

NameError: name 'df' is not defined

In [14]:
lcp = lc.plot(kind = 'bar', figsize = (20,8), width = 0.8, color = collcol) #using the yellow color for prop damage, red for injury and the defaults burn my eyes
plt.title = 'Light Condition and Severity of Crashes'
#plt.legend(loc = 2, fontsize = 14)

for p in lcp.patches:
    width = p.get_width()
    height = p.get_height()
    x,y = p.get_xy()
    val = height
    lcp.annotate(val, (x+width/2, y+height*1.02), ha= 'center')

NameError: name 'lc' is not defined

From the bar chart we can see that with the exception of the 'Unknown' light condition category 

In [15]:
stuff = df[['SEVERITYCODE', 'ROADCOND', 'LIGHTCOND' , 'WEATHER']]
stuff.head()

NameError: name 'df' is not defined

In [16]:
"""#trying some other stuff out
stuffv = stuff.valuecounts()
sp = stuffv.plot(kind = 'bar', figsize = (20,8), width = 0.8, color = ('#FFC416', '#FF3B00', 'blue'))
#plt.title = 'Light Condition and Severity of Crashes'
#plt.legend(loc = 2, fontsize = 14)

for p in sp.patches:
    width = p.get_width()
    height = p.get_height()
    x,y = p.get_xy()
    val = height
    sp.annotate(val, (x+width/2, y+height*1.02), ha= 'center')"""
print("I'll work on this later")

I'll work on this later


In [17]:
feat = df[['ROADCOND', 'LIGHTCOND' , 'WEATHER']]
feat.head()

NameError: name 'df' is not defined

In [18]:
x = feat
x[0:5]

NameError: name 'feat' is not defined

In [19]:
y = df['SEVERITYDESC'].values
y[0:5]

NameError: name 'df' is not defined

## Person count

In [20]:
pc = df['PERSONCOUNT'].value_counts()
pc

NameError: name 'df' is not defined

In [21]:
pc.plot(type = 'Box', figsize = (6,8))
plt.y_label('Number of People')
plt.show()

NameError: name 'pc' is not defined

## Map View

In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: / 

In [ ]:
crashloc = df[['X', 'Y']]
crashloc = crashloc.dropna()
crashloc.head()
#crashloc = crashloc[:1000] #limiting the length to save time for now
#print(crashloc.X)

In [ ]:
# define the world map
crashmap = folium.Map(location = [47.606, -122.332], zoom_start = 11)

# display world map
#crashmap #commenting out because i don't need a massive map showing up all the time

In [ ]:
'''crash = folium.map.FeatureGroup()
for lat, lng, in zip(crashloc.Y, crashloc.X):
    crash.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

crashmap.add_child(crash)
print('done!')'''

In [ ]:
from folium import plugins

crashmap = folium.Map(location = [47.606, -122.332], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(crashmap)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, in zip(crashloc.Y, crashloc.X):
    folium.Marker(
        location=[lat, lng],
        icon=None,
    ).add_to(incidents)

# display map
crashmap

In [ ]:
#Old code bits I don't think I need but It'd rather not delete the code outright incase i need it

#decision tree stuff
'''from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
print('Decision tree installed!')

x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=4)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)

x= preprocessing.StandardScaler().fit(x).transform(x)
x[0:5]

decTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
decTree.fit(x_train,y_train)

predict = decTree.predict(x_test)
print(predict[0:5])
print(y_test[0:5])'''